In [72]:
using Pkg
Pkg.activate(".")
using CSV: read
using DataFrames
using TidierData
using GLM

  Activating project at `~/workspace/kaggle/playground-series-s4e2`


In [56]:
test_df = read("test.csv", DataFrame)
train_df = read("train.csv", DataFrame)

@glimpse(df)

Rows: 20758
Columns: 18
.id            Int64          0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 
.Gender        CategoricalArrays.CategoricalValue{String7, UInt32}Male, Female, 
.Age           Float64        24.443011, 18.0, 18.0, 20.952737, 31.641081, 18.12
.Height        Float64        1.699998, 1.56, 1.71146, 1.71073, 1.914186, 1.7485
.Weight        Float64        81.66995, 57.0, 50.165754, 131.274851, 93.798055, 
.family_history_with_overweightCategoricalArrays.CategoricalValue{String3, UInt3
.FAVC          CategoricalArrays.CategoricalValue{String3, UInt32}yes, yes, yes,
.FCVC          Float64        2.0, 2.0, 1.880534, 3.0, 2.679664, 2.919751, 1.991
.NCP           Float64        2.983297, 3.0, 1.411685, 3.0, 1.971472, 3.0, 3.0, 
.CAEC          CategoricalArrays.CategoricalValue{String15, UInt32}Sometimes, Fr
.SMOKE         CategoricalArrays.CategoricalValue{String3, UInt32}no, no, no, no
.CH2O          Float64        2.763573, 2.0, 1.910378, 1.674061, 1.979848, 2.137
.SCC

.FAF           Float64        0.0, 1.0, 0.866045, 1.467863, 1.967973, 1.930033, 
.TUE           Float64        0.976473, 1.0, 1.673584, 0.780199, 0.931721, 1.0, 
.CALC          CategoricalArrays.CategoricalValue{String15, UInt32}Sometimes, no
.MTRANS        CategoricalArrays.CategoricalValue{String31, UInt32}Public_Transp
.NObeyesdad    CategoricalArrays.CategoricalValue{String31, UInt32}Overweight_Le


In [57]:
coerce!(train_df,
    :id => Count,
    :Gender => Multiclass,
    :Age => Continuous,
    :Height => Continuous,
    :Weight => Continuous,
    :family_history_with_overweight => Multiclass,
    :FAVC => Multiclass,
    :FCVC => Continuous,
    :NCP => Continuous,
    :CAEC => Multiclass,
    :SMOKE => Multiclass,
    :CH2O => Continuous,
    :SCC => Multiclass,
    :FAF => Continuous,
    :TUE => Continuous,
    :CALC => Multiclass,
    :MTRANS => Multiclass,
    :NObeyesdad => OrderedFactor
); MLJ.schema(train_df)

┌────────────────────────────────┬──────────────────┬───────────────────────────
│ names                          │ scitypes         │ types                    ⋯
├────────────────────────────────┼──────────────────┼───────────────────────────
│ id                             │ Count            │ Int64                    ⋯
│ Gender                         │ Multiclass{2}    │ CategoricalValue{String7 ⋯
│ Age                            │ Continuous       │ Float64                  ⋯
│ Height                         │ Continuous       │ Float64                  ⋯
│ Weight                         │ Continuous       │ Float64                  ⋯
│ family_history_with_overweight │ Multiclass{2}    │ CategoricalValue{String3 ⋯
│ FAVC                           │ Multiclass{2}    │ CategoricalValue{String3 ⋯
│ FCVC                           │ Continuous       │ Float64                  ⋯
│ NCP                            │ Continuous       │ Float64                  ⋯
│ CAEC                      

In [67]:
train_X = @select(train_df, !id, !NObeyesdad);
train_y = @select(train_df, NObeyesdad);
test_X = @select(test_df, !id);

In [86]:
multiclass_cols = [:Gender, :family_history_with_overweight, :FAVC, :CAEC, :SMOKE, :SCC, :CALC, :MTRANS]

onehot = OneHotEncoder(features=multiclass_cols)

MLJ.transform(onehot, train_X)


MethodError: MethodError: no method matching transform(::OneHotEncoder, ::DataFrame)

Closest candidates are:
  transform(::OneHotEncoder, ::Any, !Matched::Any)
   @ MLJModels ~/.julia/packages/MLJModels/iFOi9/src/builtins/Transformers.jl:934
  transform(!Matched::UnivariateStandardizer, ::Any, !Matched::Real)
   @ MLJModels ~/.julia/packages/MLJModels/iFOi9/src/builtins/Transformers.jl:519
  transform(!Matched::UnivariateStandardizer, ::Any, !Matched::Any)
   @ MLJModels ~/.julia/packages/MLJModels/iFOi9/src/builtins/Transformers.jl:525
  ...


In [ ]:

# Create a dictionary to store the encoders
encoders = Dict{Symbol, OneHotEncoder}()

# Create and fit the encoders for each multiclass column
for col in multiclass_cols
    encoder = OneHotEncoder()
    fit!(encoder, train_X[:, col])
    encoders[col] = encoder
end

# Apply the encoders to the multiclass columns
for col in multiclass_cols
    transform!(encoders[col], train_X[:, col])
end

In [78]:
XGBoostClassifier = @load XGBoostClassifier
model = XGBoostClassifier()

import MLJXGBoostInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/alex/.julia/packages/MLJModels/iFOi9/src/loading.jl:159


XGBoostClassifier(
  test = 1, 
  num_round = 100, 
  booster = "gbtree", 
  disable_default_eval_metric = 0, 
  eta = 0.3, 
  num_parallel_tree = 1, 
  gamma = 0.0, 
  max_depth = 6, 
  min_child_weight = 1.0, 
  max_delta_step = 0.0, 
  subsample = 1.0, 
  colsample_bytree = 1.0, 
  colsample_bylevel = 1.0, 
  colsample_bynode = 1.0, 
  lambda = 1.0, 
  alpha = 0.0, 
  tree_method = "auto", 
  sketch_eps = 0.03, 
  scale_pos_weight = 1.0, 
  updater = nothing, 
  refresh_leaf = 1, 
  process_type = "default", 
  grow_policy = "depthwise", 
  max_leaves = 0, 
  max_bin = 256, 
  predictor = "cpu_predictor", 
  sample_type = "uniform", 
  normalize_type = "tree", 
  rate_drop = 0.0, 
  one_drop = 0, 
  skip_drop = 0.0, 
  feature_selector = "cyclic", 
  top_k = 0, 
  tweedie_variance_power = 1.5, 
  objective = "automatic", 
  base_score = 0.5, 
  watchlist = nothing, 
  nthread = 8, 
  importance_type = "gain", 
  seed = nothing, 
  validate_parameters = false, 
  eval_metric = String

In [79]:
mach = machine(model, train_X, train_y)

┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc XGBoost.XGBoostClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Count}, AbstractVector{Multiclass{4}}, AbstractVector{Multiclass{2}}, AbstractVector{Multiclass{3}}, AbstractVector{Multiclass{5}}, AbstractVector{OrderedFactor{7}}}}, Table{AbstractVector{OrderedFactor{7}}}}
│ 
│ fit_data_sci

untrained Machine; caches model-specific representations of data
  model: XGBoostClassifier(test = 1, …)
  args: 
    1:	Source @806 ⏎ Table{Union{AbstractVector{Continuous}, AbstractVector{Count}, AbstractVector{Multiclass{4}}, AbstractVector{Multiclass{2}}, AbstractVector{Multiclass{3}}, AbstractVector{Multiclass{5}}, AbstractVector{OrderedFactor{7}}}}
    2:	Source @577 ⏎ Table{AbstractVector{OrderedFactor{7}}}
